# Olympics CSV Data Cleaning

In [1]:
# snippet to download the data file the first time
import os.path  # library for working with local directories

filename = 'olympics.csv'
url = r"https://github.com/realpython/python-data-cleaning/raw/master/Datasets/olympics.csv"

if not os.path.isfile(filename):  # if filename doesn't exist in current directory...
    import urllib.request  # library for making web requests
    
    # Download the file from `url` and save it locally under `filename`:
    urllib.request.urlretrieve(url, filename)
    print(filename, 'downloaded')
    
else:
    print(filename, 'already present')

olympics.csv already present


The new CSV file should contain the following columns:

0) `entry` - entry number (start at 1)

1) `name` - country/team name, with no codes/abbreviations

2) `code` - 3-letter/number country code, no parentheses

3) `01 ! combined` - # first place finishes/gold medals in summer + winter combined

4) `02 ! combined` - silver

5) `03 ! combined` - bronze

6) `? games` - total number of olympics (summer + winter combined)

7) `winter` - *no* if the team has never played in a Winter Olympics, *yes* otherwise

(8 columns in all)
 
* teams should be sorted by their 3-letter/number code, *not* their full name


Looking at a correct output file in a plain text editor, the first two rows should be as follows:

`entry,name,code,01 ! combined,02 ! combined,03 ! combined,? games,winter
1,Afghanistan,AFG,0,0,2,13,False`

In [7]:
import pandas as pd
import re
pd.options.mode.chained_assignment = None # default='warn'

def process_olympics_data(input_filename, output_filename='output.csv'):
    """Modifies the given 'olympics.csv' file into the specifications laid out above.
       
       Input: This def takes the name of the file to be modified, and the user specifies the name of the file it will output
       
       Returns: A csv file of the given specifications
    """
    #sets the columns from the original csv that we care about
    col_list = ["0","6","11","12","13","14"]
    #gives a new column name to importanted columns since we are removing the headers on import
    new_col_list = ["name","winter","? games","01 ! combined","02 ! combined","03 ! combined"]
    #import and read the given csv file, uses the above objects to exactract only what is needed
    csvf = pd.read_csv(input_filename, skiprows=2, skipfooter=1, usecols=[0,6,11,12,13,14], names=new_col_list)
    csvf["entry"] = range(1, len(csvf)+1)              #adds "entry" column and gives it a range
    csvf["code"] = range(1, len(csvf)+1)               #adds "code" column and gives it a notional range, replaced later

    a = csvf['name']                                   #picks a column to use as an index for the loop, could be any column
    for i in range(len(csvf)):
        a_it = a[i]                                    #recreates a workable object each iteration
        b_it = a_it.split(" [",1)[0]                   #splits each entry so that nothing after the country code is used

        inp = re.findall((r"\((.*?)\)"), b_it)         #searches and displays the country code from inside the parenthesis
        csvf["code"][i]=inp[0]                         #places the country code in the code column

        inp2 = re.sub(r"(.*?)\s?\(.*?\)", r"\1", b_it) #searches and displays the country name
        csvf["name"][i]=inp2[0:len(inp2)]              #places the country name in the names column

        if int(csvf['winter'][i])>0:                   #this if/else replaces the winter numbers with no or yes based on
            csvf['winter'][i] = "True"                 #whether the number was 0 or not.
        else:                                          
            csvf['winter'][i] = "False"

        #rearrange the columns into the proper order
        output = csvf[["entry","name","code","01 ! combined","02 ! combined","03 ! combined","? games","winter"]]
    return output.to_csv('output.csv', index=False)

In [8]:
process_olympics_data('olympics.csv')
#Output file will be in the same directory as this file